## Code playground for the lecture block *Random numbers*

In the following, we will explore some of the concepts discussed in the lecture using simulations. Some of the code will have missing parts that you have to fill in. You can also change parts of the code yourself to play around with it.

### 1. Bayes theorem

As a warm-up, we will have a closer look at Bayes Theorem. Below is a function 'simulate_test_development' that simulates testing of a group of subjects for a sickness.

The function returns the probability that if the test is positive, the subject is actually sick.

**Your task is to complete the function 'bayes' given sensitivity, specificity, and the general incidence (p(sick)), and compare the result with the simulation for various parameters.**

In [ ]:
# Import the packages we need
import numpy as np

# First, we set the random seed. This way, we will always produce the same random numbers when we run the code.
# This is useful, as our experiment becomes reproducible this way!
np.random.seed(424242)

def simulate_test_development(sensitivity, specificity, incidence_of_infection, number_of_test_subjects):
    # We have 'number_of_test_subjects' people in our test population.
    # According to the incidence of infection, we randomly assign whether a person is infected or not.
    ground_truth = np.random.binomial(1, incidence_of_infection, number_of_test_subjects)
    # We sort the array for convenience
    ground_truth = np.sort(ground_truth)

    # We test the sick subjects
    test_result_true_positive = np.random.binomial(1, sensitivity, np.sum(ground_truth))
    # We test the healthy subjects
    test_result_true_negative = 1-np.random.binomial(1, specificity, number_of_test_subjects - np.sum(ground_truth))

    # We concatenate the results to get the test results of each subject
    test_results = np.concatenate((test_result_true_negative, test_result_true_positive))

    # From the ground truth and the test results, we can calculate p(subject is sick | test is positive)
    prob_sick_if_positive = np.sum(ground_truth[test_results==1])/len(test_results[test_results==1])

    return prob_sick_if_positive
    
def bayes(sensitivity, specificity, incidence_of_infection):
    ## YOUR CODE GOES HERE
    prior =  # p(sick)
    evidence = # p(positive)
    likelihood_fct = # p(test is positive | subject is sick)/p(positive)
    posterior = # Bayes theorem
    return posterior

In [ ]:
sensitivity = .975 # true positive rate
specificity = .975 # true negative rate

incidence_of_infection = 0.001 # #infections/#population
number_of_test_subjects = 10000 # number of subjects in the test study

## Run this to get the results! Try out different numbers for sensitivity, specificity, incidence, and number of test subjects!
print('Simulation: p(subject is sick | test is positive) = {}'.format(simulate_test_development(sensitivity, specificity, incidence_of_infection, number_of_test_subjects)))
print('Bayes theorem: p(subject is sick | test is positive) = {}'.format(bayes(sensitivity, specificity, incidence_of_infection)))


### 2. Probability distributions

Lets have a look at random variables from different probability distributions. Here, we are looking at the binomial and the Poisson distribution.

We start with the binomial distribution. The binomial distribution describes a sequence of random events, where each random event has two outcomes: one with probability $p$, and one with probability $1-p$.

A simple example is tossing a coin, where $p = 0.5$. The binomial distribution $P(k\ |\ n, p) = \binom{n}{k} p^k (1-p)^{n-k}$ gives us the probability that, when tossing $n$ coins, we find exactly $k$ times heads.

**In the following, play around with the two parameters repeats ($n$) and prob ($p$). What happens for very large values of $n$? Does this remind you of another distribution you know?**

In [ ]:
import numpy as np
# matplotlib and seaborn are used for visualisation
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(424242)

# Set up the parameters
prob = 0.5
repeats = 10
num_samples = 100000

# Sample 'num_samples' from a binomial distribution
samples = np.random.binomial(repeats, prob, num_samples)

# visualise the samples
_ = plt.hist(samples, bins = 50, density=True, edgecolor = 'k', linewidth=0.5)
plt.ylabel(r'$p(x)$', fontsize = 18, rotation = 0, labelpad=30)
plt.xlabel(r'$x$', fontsize = 18)
plt.title('Binomial distribution', fontsize = 18)
plt.tick_params(axis='both', which='major', labelsize=18)
sns.despine()

Let's continue with another common probability distribution: the Poisson distribution. Assume we have certain events that can occur at any time, with an average rate of occurence $\lambda > 0$. For example, this could be incoming phone calls in a call centre, radioactive decay of isotopes in a material, the number of spikes a cortical neuron receives from other neurons, etc. 

The Poisson distribution $p(k\ |\ \lambda) = \frac{\lambda^k e^{-\lambda}}{k!}$ yields the probability that in a given time interval, we observe $k$ such events. 

**What happens for large rates? Does the distribution seem familiar?**

In [ ]:
import numpy as np
# matplotlib and seaborn are used for visualisation
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(424242)

# Set up the parameters
lambdas = 1
num_samples = 100000

# Sample 'num_samples' from a Poisson distribution
samples = np.random.poisson(lambdas, num_samples)

# visualise the samples
_ = plt.hist(samples, bins = 30, density=True, edgecolor = 'k', linewidth=0.5)
plt.ylabel(r'$p(x)$', fontsize = 18, rotation = 0, labelpad=30)
plt.xlabel(r'$x$', fontsize = 18)
plt.title('Poisson distribution', fontsize = 18)
plt.tick_params(axis='both', which='major', labelsize=18)
sns.despine()

**Curiosity:** During WW2, the Poisson distribution was used by British statisticians to identify that Nazi Germany did not possess advanced missile guidance systems, i.e., they dropped bombs randomly.
See http://doi.org/10.1111/j.1740-9713.2019.01315.x

### 3. Concentration bounds

In this part, we will have a look at concentration bounds: Law of Large Numbers (LLN) and Chernoff Bounds.

First, we explore this for a normal distribution with $\mu = 0$ and $\sigma^2 = 1$. We sample the mean for different numbers of $n$ (number of samples used to calculate the arithmetic mean), and compare our result with the above bounds.

Moreover, we look at the distribution of the mean for $n=1$ and the maximum value of $n$ (here: $n = 2000$). As you will see, this distribution becomes narrower for increasing $n$!

**Your task is to complete the function 'get_prob_that_mean_larger_epsilon', which calculates $p(|mean - E(x)| \geq \epsilon)$ from sampled data.**

In [ ]:
np.random.seed(424242)

def LLN(epsilon, num_samples, var = 1):
    return var/(epsilon**2*num_samples)

def Chernoff(epsilon, num_samples, var=1):
    return 2*np.exp(-epsilon**2*num_samples/(2*var))

# Set up the parameters
# Epsilon is the deviation from the mean
epsilon = 0.1

# Number of repeats for estimating probabilities
repeats = 20000
# Number of samples for each repeat. We are interested in the behaviour for different values.
nvals = np.arange(1, 2001, 100)

# This will be used to collect our experimental results for p(mean >= epsilon | num_samples)
probabilities = []

def get_prob_that_mean_larger_epsilon(samples, epsilon):
    ## YOUR CODE GOES HERE
    # Take the mean over the second dimension (the mean for each repeat)
    mean = 
    # Check if the absolute value of the mean exceeds epsilon. This returns a list of booleans (True: exceeds, False: does not exceed)
    check_if_larger_epsilon = 
    # To get the probability of exceeding the threshold, we take the mean of the boolean list (assuming True = 1, False = 0)
    prob_to_be_larger = 
    return prob_to_be_larger, mean

means = []
for num_samples in nvals:
    # sample (repeats, num_samples) values from a normal distribution with mean 0 and variance 1
    # note: the first dimension (repeats) is used to estimate the probability of the mean exceeding epsilon, 
    # that is, we repeat the random experiment epsilon times. You could also use a loop here.
    samples = np.random.normal(0,1, (repeats, num_samples))
    prob_to_be_larger, sampled_means = get_prob_that_mean_larger_epsilon(samples, epsilon)
    means.append(sampled_means)
    probabilities.append(prob_to_be_larger)

# visualisation of the probability that the arithmetic mean exceeds the expectation value by epsilon
fig, ax = plt.subplots()
plt.plot(nvals, LLN(epsilon, nvals), label = 'LLN', linewidth=4, linestyle='--', color = 'tomato')
plt.plot(nvals, Chernoff(epsilon, nvals), label = 'Chernoff', linewidth=4, linestyle='dotted', color = 'steelblue')
plt.plot(nvals, probabilities, label = 'Simulation', linewidth=4, color = 'k')
ax.tick_params(axis='both', which='major', labelsize=18)
plt.ylabel(r'$p\left(\left|\frac{1}{n} \sum_{i=1}^n x_i - \text{E}(x_1)\right| \geq \epsilon\right)$', fontsize = 18, labelpad=10)
plt.xlabel(r'$n$', fontsize = 18)
plt.legend(frameon=False, fontsize=18)
plt.yscale('log')
sns.despine()

# visualisation of the distribution of means
fig, ax = plt.subplots(1,2, figsize=(12,6))

ax[0].hist(means[0], bins = np.arange(-5,5, 0.5), density=True, edgecolor = 'k', linewidth=0.5)
ax[0].set_xlabel(r'arithmetic mean $\bar{x}_n$ for $n=1$', fontsize = 18)
ax[0].set_ylabel(r'$p(\bar{x}_n)$', fontsize = 18)
ax[0].tick_params(axis='both', which='major', labelsize=18)
ax[1].hist(means[-1], bins = np.arange(-5,5, 0.05), density=True, edgecolor = 'k', linewidth=0.5)
ax[1].set_xlabel(r'arithmetic mean $\bar{x}_n$ for $n=2000$', fontsize = 18)
ax[1].tick_params(axis='both', which='major', labelsize=18)
sns.despine()

Lets apply our code to the Cauchy distribution. As you can see... the distribution of means does not change for increasing $n$!

In [ ]:
# Number of repeats for estimating probabilities
repeats = 20000
# Number of samples for each repeat. We are interested in the behaviour for different values.
nvals = np.arange(1, 2001, 100)

means = []
for num_samples in nvals:
    # sample (repeats, num_samples) values from a normal distribution with mean 0 and variance 1
    # note: the first dimension (repeats) is used to estimate the probability of the mean exceeding epsilon, 
    # that is, we repeat the random experiment epsilon times. You could also use a loop here.
    samples = np.random.standard_cauchy((repeats, num_samples))
    prob_to_be_larger, sampled_means = get_prob_that_mean_larger_epsilon(samples, epsilon)
    means.append(sampled_means)

# visualisation of the distribution of means
fig, ax = plt.subplots(1,2, figsize=(12,6))

ax[0].hist(means[0], bins = np.arange(-10,10, 0.5), density=True, edgecolor = 'k', linewidth=0.5)
ax[0].set_xlabel(r'arithmetic mean $\bar{x}_n$ for $n=1$', fontsize = 18)
ax[0].set_ylabel(r'$p(\bar{x}_n)$', fontsize = 18)
ax[0].tick_params(axis='both', which='major', labelsize=18)
ax[1].hist(means[-1], bins = np.arange(-10,10, 0.5), density=True, edgecolor = 'k', linewidth=0.5)
ax[1].set_xlabel(r'arithmetic mean $\bar{x}_n$ for $n=2000$', fontsize = 18)
ax[1].tick_params(axis='both', which='major', labelsize=18)

### 4. Inverse transform sampling

In the lecture, we saw what happens to the probability distribution of a random variable $x$ if we apply an invertible function $f(x)$.

In this exercise, we will turn this process around! Assume we have a random variable $x$ sampled from a uniform distribution. How do we have to choose $f$ such that $f(x)$ follows a desired probability distribution $p$?

A technique solving this problem is 'inverse transform sampling':

1) Calculate the cumulative distribution of $p$, denoted by $P(x') = \int_{a}^{x'} p(x) dx$, where $a$ is the lower bound of the distribution. We choose $f = P^{-1}$.
2) Generate a random number $x$ from the uniform distribution $U(0,1)$.
3) Calculate $y = f(x)$. $y$ follows the probability distribution $p$.

Why? Because we have
- $x = f^{-1}(y) = P(y) \leftrightarrow \frac{dx}{dy} = p(y) \leftrightarrow \int 1 dx = \int \frac{dx}{dy} dy = \int p(y) dy$

**Meaning:** if x follows the uniform distribution (in $[0,1]$), then $y$ follows $p(y)$. The last step is exactly what we did in the lecture when transforming random variables.

**We will try this for the exponential distribution** $p(y) = \lambda e^{-\lambda y}$ with $\lambda > 0$ and $y \geq 0$.

What you have to do:

Step 1: Calculate the cumulative distribution $P(x') = \int_0^{x'} p(x) dx$

Step 2: Invert $P$ to get $f = P^{-1}$. Complete 'function_f' using this expression.

Step 3: Sample random numbers from a uniform distribution (number between 0 and 1) and apply $f$. This part is already implemented.

Step 4: Compare the result with the exponential distribution (the red line in the figure).

In [ ]:
def function_f(x, lambdas):
    # YOUR CODE GOES HERE
    # return the expression for f
    return 

def exponential_distribution(x, lambdas):
    return lambdas*np.exp(-lambdas*x)

samples = np.random.uniform(size=(10000))
transformed_samples = function_f(samples, 1)

xvals = np.linspace(0, np.max(transformed_samples), 1000)
plt.hist(function_f(samples, 1), bins = 50, density=True, color = 'steelblue', edgecolor = 'k', linewidth=0.5)
plt.plot(xvals, exponential_distribution(xvals, 1), linewidth = 4, color = 'tomato')
plt.xlabel(r'$y$', fontsize = 18)
plt.ylabel(r'$p(y)$', fontsize = 18)
plt.tick_params(axis='both', which='major', labelsize=18)
sns.despine()

### 5. Central limit theorem

The last example is the famous central limit theorem. We will look at it for the uniform distribution first.

**Below is a setup for exploring the central limit theorem. Your task is to complete the function 'get_sum_of_uniforms'. The remaining code creates a histogram of the sum of random variables you obtained, and compares it with a Gaussian with zero mean and unit variance (red line)**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

np.random.seed(424242)

num_of_rv_to_sum = 5

def func(x, mu=0, sigma=1):
    # Unit Gaussian function, used for visualisation.
    return 1/np.sqrt(2*np.pi*sigma**2) *np.exp(-(x-mu)**2/(2*sigma**2))

def get_sum_of_uniforms(num_of_rv_to_sum, repeats = 10000):
    ## YOUR CODE GOES HERE
    # What we need is the following:
    # repeat times, we sample 'num_of_rv_to_sum' numbers from a uniform distribution, 
    # sum them, and divide by the square root of num_of_rv_to_sum
    # The output is a list of length 'repeats', i.e., we sample the sum 'repeats' times
    
    return 

# Get 'repeats' samples of the sum of random variables
# We sum 'num_of_rv_to_sum' many random variables
summed_rvs = get_sum_of_uniforms(num_of_rv_to_sum)
# We center and rescale the final distribution, so that it has mean 0 and variance 1
summed_rvs = summed_rvs - 0.5*np.sqrt(num_of_rv_to_sum)
summed_rvs = summed_rvs/np.sqrt((1/12))

# Visualise the distribution
xval = np.linspace(-10, 10, 10000)
_ = plt.hist(summed_rvs, bins = 100, density=True, color='steelblue', label = 'Histogram', edgecolor = 'k', linewidth=0.5)
plt.plot(xval, func(xval), color = 'tomato', linewidth=2.5, label = 'Theory')
plt.tick_params(axis='both', which='major', labelsize=18)
sns.despine()
plt.xlabel('x', fontsize = 20)
plt.ylabel('p(x)', fontsize = 20, rotation = 0, labelpad=20)
plt.xlim(-3,3)

The uniform distribution is already symmetric, but what happens for a distribution that is not symmetric? A good example is the exponential distribution!

**Your task: implement the same function as before, but now sample the random variables from the exponential distribution. What happens for increasing values of 'num_of_rv_to_sum'? Can you explain this?**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

np.random.seed(424242)

num_of_rv_to_sum = 1
lambdas = 2

def func(x, mu=0, sigma=1):
    # Unit Gaussian function
    return 1/np.sqrt(2*np.pi*sigma**2) *np.exp(-(x-mu)**2/(2*sigma**2))

def get_sum_of_exponentials(num_of_rv_to_sum, repeats = 10000):
    ## YOUR CODE GOES HERE
    # Lets do the same, but for the exponential distribution
    # It is given by np.random.exponential and has the 'scale' parameter,
    # which we set to 1/lambdas (the mean of the distribution)
    
    return

# Get 'repeats' samples of the sum of random variables
# We sum 'num_of_rv_to_sum' many random variables
summed_rvs = get_sum_of_exponentials(num_of_rv_to_sum)
# We center and rescale the final distribution, so that it has mean 0 and variance 1
summed_rvs = summed_rvs - 1/lambdas*np.sqrt(num_of_rv_to_sum)
summed_rvs = summed_rvs/np.sqrt((1/lambdas**2))

# Visualise the distribution
xval = np.linspace(-10, 10, 10000)
_ = plt.hist(summed_rvs, bins = 100, density=True, color='steelblue', label = 'Histogram', edgecolor = 'k', linewidth=0.5)
plt.plot(xval, func(xval), color = 'tomato', linewidth=2.5, label = 'Theory')
plt.tick_params(axis='both', which='major', labelsize=18)
sns.despine()
plt.xlabel('x', fontsize = 20)
plt.ylabel('p(x)', fontsize = 20, rotation = 0, labelpad=20)
plt.xlim(-3,3)

What happens if we use the Cauchy distribution instead...?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

np.random.seed(424242)

num_of_rv_to_sum = 1

def func(x, mu=0, sigma=1):
    # Unit Gaussian function
    return 1/np.sqrt(2*np.pi*sigma**2) *np.exp(-(x-mu)**2/(2*sigma**2))

def get_sum_of_cauchys(num_of_rv_to_sum, repeats = 10000):
    ## YOUR CODE GOES HERE
    # Lets do the same, but for the cauchy distribution

    return 

# Get 'repeats' samples of the sum of random variables
# We sum 'num_of_rv_to_sum' many random variables
summed_rvs = get_sum_of_cauchys(num_of_rv_to_sum)
# There is no defined mean or variance, so lets try centering using the observed mean and variance!
summed_rvs = summed_rvs - np.mean(summed_rvs)
summed_rvs = summed_rvs/np.std(summed_rvs)

# Visualise the distribution
xval = np.linspace(-10, 10, 10000)
_ = plt.hist(summed_rvs, bins = 2000, density=True, color='steelblue', label = 'Histogram', edgecolor = 'k', linewidth=0.5)
plt.plot(xval, func(xval), color = 'tomato', linewidth=2.5, label = 'Theory')
plt.tick_params(axis='both', which='major', labelsize=18)
sns.despine()
plt.xlabel('x', fontsize = 20)
plt.ylabel('p(x)', fontsize = 20, rotation = 0, labelpad=20)
plt.xlim(-3,3)

### 6. Real data

To close this notebook, we take a brief look at actual data. In the following, we load data containing features extracted using recordings from the Hubble Space Telescope for sources (e.g., globular clusters, stars, background galaxies, etc.) in the galaxy 'NGC1427a'.

Basically, we record the galaxy for two different frequency bands (i.e., we have two images of it, in the g and z band). After identifying the sources, we create small cut-outs. Thus, for every source we have two small images (the flux for each band).

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import pandas as pd

dframe = pd.read_csv('NGC1427a_sources.csv')
images = fits.getdata('NGC1427a_images.fits')

fig, ax = plt.subplots(1,2, figsize=(12,12))
ax[0].imshow(images[115][0])
ax[0].set_title('g-band image', fontsize = 20)
ax[0].axis('off')
ax[1].imshow(images[115][1])
ax[1].set_title('z-band image', fontsize = 20)
ax[1].axis('off')

We then preprocess the data by extracting useful information from the images, such as

- features that describe the shape (eccentricity, orientation, area, ...)
- features that describe the flux (e.g., m3_g = brightness using a 3 pixel aperture in the g-band)

Thus, we obtain a table containing several features for each source we identified. Just for fun, lets have a look at m3_g and m3_z.

In [ ]:
import seaborn as sns

plt.scatter(dframe['m3_g'], dframe['m3_z'], alpha = 0.5, color = 'steelblue', edgecolors='k')
plt.ylabel('m3_z', fontsize = 20, rotation = 0, labelpad=35)
plt.xlabel('m3_g', fontsize = 20)
plt.tick_params(axis='both', which='major', labelsize=18)
sns.despine()

It looks like the data follows some distribution. Lets try a multivariate Gaussian. We fit a 2D Gaussian using 'GaussianMixture' to the data, generate new samples from it, and compare the original data and our 'model-generated data' visually.

In [ ]:
from sklearn.mixture import GaussianMixture

fig, ax = plt.subplots(1,2, figsize=(12,6), sharex=True, sharey=True)

gmm = GaussianMixture(n_components=1).fit(dframe[['m3_g', 'm3_z']].values)
model_samples, _ = gmm.sample(len(dframe))
ax[0].scatter(dframe['m3_g'], dframe['m3_z'], alpha = 0.5, color = 'steelblue', edgecolors='k')
ax[0].set_title('Original data', fontsize = 20)
ax[1].scatter(model_samples[:,0], model_samples[:,1], color = 'tomato', alpha = 0.5, marker = 'o', edgecolor = 'k')
ax[1].set_title('Data from single Gaussian', fontsize = 20)
ax[0].tick_params(axis='both', which='major', labelsize=18)
ax[1].tick_params(axis='both', which='major', labelsize=18)
sns.despine()

Instead of assuming that the underlying data distribution is a single Gaussian, we can model the distribution using a weighted sum of Gaussians to be more flexible (an intuition: we can imagine that we have different types of sources (stars, galaxies, globular clusters, etc.) which follow different Gaussians!). You see the result below, which doesn't look too bad!

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,6), sharex=True, sharey=True)

gmm = GaussianMixture(n_components=6).fit(dframe[['m3_g', 'm3_z']].values)
model_samples, _ = gmm.sample(len(dframe))
ax[0].scatter(dframe['m3_g'], dframe['m3_z'], alpha = 0.5,  color = 'steelblue', edgecolors='k')
ax[0].set_title('Original data', fontsize = 20)
ax[1].scatter(model_samples[:,0], model_samples[:,1], color = 'tomato', alpha = 0.5, marker = 'o', edgecolor = 'k')
ax[1].set_title('Data from mixture of Gaussians', fontsize = 20)
ax[0].tick_params(axis='both', which='major', labelsize=18)
ax[1].tick_params(axis='both', which='major', labelsize=18)
sns.despine()

Of course we only selected two out of many features here, but I hope this illustrates our assumption that data $x$ is generated from some underlying distribution $p(x)$. Our task is then to model this distribution using a parametrised function $f(x | \theta)$ using parameters $\theta$ (in case of a Gaussian mixture model, its the parameters of the Gaussians and the weighting factors of each Gaussian).